In [ ]:
%pip install numpy
%pip install nltk
%pip install wikipedia
%pip install scikit-learn

In [ ]:

import nltk
import wikipedia
import random
import string
import numpy as np
import io
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
